# Walk forward optimization

Reference 1: https://wire.insiderfinance.io/advanced-backtesting-on-nifty-using-walk-forward-analysis-in-python-90da743fcf78

Reference 2: https://pyquantnews.com/1000000-backtest-simulations-20-seconds-vectorbt/

Reference 3: https://algotrading101.com/learn/walk-forward-optimization/

Reference 4 - Interpreting Walk forward Optimization reuslt: https://www.tradelikeamachine.com/backtesting-software-walk-forward-pro/user-guide/interpreting-walk-forward-results

In [41]:
import numpy as np
import scipy.stats as stats
import vectorbt as vbt

In [42]:
windows = np.arange(10, 50)
price = vbt.YFData.download('AAPL').get('Close')

In [43]:
(in_price, in_indexes), (out_price, out_indexes) = price.vbt.rolling_split(
    n=30, 
    window_len=365 * 2,
    set_lens=(180,),
    left_to_right=False,
)
# Here n = 30 means we are dividing our total datapoints into 30 different windows, with each window lenght *2 from which
# 450 are insample data and 50 are out of the sample data

In [44]:
def simulate_all_params(price, windows, **kwargs):
    fast_ma, slow_ma = vbt.MA.run_combs(
        price, windows, r=2, short_names=["fast", "slow"]
    )
    entries = fast_ma.ma_crossed_above(slow_ma)
    exits = fast_ma.ma_crossed_below(slow_ma)

    pf = vbt.Portfolio.from_signals(price, entries, exits, **kwargs)
    return pf.sharpe_ratio()

In [47]:
def simulate_best_params(price, best_fast_windows, best_slow_windows, **kwargs):
    
    fast_ma = vbt.MA.run(price, window=best_fast_windows, per_column=True)
    slow_ma = vbt.MA.run(price, window=best_slow_windows, per_column=True)

    entries = fast_ma.ma_crossed_above(slow_ma)
    exits = fast_ma.ma_crossed_below(slow_ma)

    pf = vbt.Portfolio.from_signals(price, entries, exits, **kwargs)
    return pf.sharpe_ratio()

In [36]:
in_sharpe = simulate_all_params(
    in_price, 
    windows, 
    direction="both", 
    freq="d"
)

In [37]:
out_test_sharpe = simulate_best_params(
    out_price, 
    in_best_fast_windows, 
    in_best_slow_windows, 
    direction="both", 
    freq="d"
)

In [49]:
in_sample_best = in_sharpe[in_best_index].values
out_sample_test = out_test_sharpe.values

t, p = stats.ttest_ind(
    a=out_sample_test,
    b=in_sample_best,
    alternative="greater"
)

In [50]:
in_best_index = get_best_index(in_sharpe)

in_best_fast_windows = get_best_params(
    in_best_index,
    'fast_window'
)
in_best_slow_windows = get_best_params(
    in_best_index,
    'slow_window'
)
in_best_window_pairs = np.array(
    list(
        zip(
            in_best_fast_windows, 
            in_best_slow_windows
        )
    )
)

In [51]:
out_test_sharpe = simulate_best_params(
    out_price, 
    in_best_fast_windows, 
    in_best_slow_windows, 
    direction="both", 
    freq="d"
)

In [52]:
in_sample_best = in_sharpe[in_best_index].values
out_sample_test = out_test_sharpe.values

t, p = stats.ttest_ind(
    a=out_sample_test,
    b=in_sample_best,
    alternative="greater"
)

In [53]:
p

0.9999965553508084